In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# 模擬數據
num_samples = 2
frames = np.random.random((num_samples, 64, 64,3))
actions = np.random.randint(3, size=num_samples)
durations = np.random.random(num_samples)
scores = np.random.randint(1, 101, size=num_samples)

# 將數據分為訓練集和驗證集
X_train, X_val, y_train_actions, y_val_actions, y_train_durations, y_val_durations, y_train_scores, y_val_scores = train_test_split(
    frames, actions, durations, scores, test_size=0.2, random_state=42
)

# 假設 actions 有3個可能的值（例如：跳躍，滑行，其他）
num_classes = 3
y_train_actions_categorical = to_categorical(y_train_actions, num_classes=num_classes)
y_val_actions_categorical = to_categorical(y_val_actions, num_classes=num_classes)

In [ ]:
#確認數據
print(frames)
print("--------------------------------------------")
print(actions)
print("--------------------------------------------")
print(durations)
print("--------------------------------------------")
print(scores)

In [ ]:
input_frames = Input(shape=(64, 64, 3), name='frames')
input_actions = Input(shape=(1,), name='actions')
input_durations = Input(shape=(1,), name='durations')

x = Conv2D(32, (3, 3), activation='relu')(input_frames)
x = Flatten()(x)

concatenated = Concatenate()([x, input_actions, input_durations])

dense = Dense(128, activation='relu')(concatenated)
score_output = Dense(1, activation='linear', name='score')(dense)

score_model = Model(inputs=[input_frames, input_actions, input_durations], outputs=score_output)
score_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

# 訓練評分預測模型
score_model.fit(
    [X_train, y_train_actions, y_train_durations],
    y_train_scores,
    validation_data=([X_val, y_val_actions, y_val_durations], y_val_scores),
    epochs=10,
    batch_size=32
)


In [ ]:
# 加載訓練好的舊模型
custom_objects = {'mse': 'mean_squared_error'}
old_model = load_model('test_cookie_run_model.h5', custom_objects=custom_objects)

# 使用新的數據進行再訓練，增加評分數據作為權重進行優化
predicted_scores = score_model.predict([X_train, y_train_actions, y_train_durations])
weights = predicted_scores.flatten() / 100.0  # 將評分標準化作為權重
# 為每個輸出分別創建sample_weight
action_weights = weights
duration_weights = weights
# 重新編譯主模型以應用新優化器
old_model.compile(optimizer=Adam(learning_rate=0.001), loss={'action_output': 'categorical_crossentropy', 'duration_output': 'mse'}, metrics={'action_output': 'accuracy', 'duration_output': 'mse'})

# 確保目標數據的形狀與模型輸出的形狀匹配
old_model.fit(
    X_train,
    {'action_output': y_train_actions_categorical, 'duration_output': y_train_durations},
    sample_weight={'action_output': action_weights, 'duration_output': duration_weights},
    validation_data=(X_val, {'action_output': y_val_actions_categorical, 'duration_output': y_val_durations}),
    epochs=10,
    batch_size=32
)

# 儲存優化後的模型
old_model.save('test_update.h5')
